In [2]:
import torch

In [15]:
# RBF kernel
def kernel__of_one_batch(Z_1, Z_2, sigma):
    length, dim = Z_1.size(0), Z_1.size(1)
    Z_1 = Z_1.reshape(length*dim)
    Z_2 = Z_2.reshape(length*dim)
    l2_norm = torch.norm(Z_1 - Z_2)*torch.norm(Z_1 - Z_2)
    kernel = torch.exp(-(l2_norm /(2*sigma)))
    return kernel

def kernel_of_different_batch(z_1, z_2, sigma):
    N = z_1.size(0)
    kernel = 0
    for i in range(N):
        for j in range(N):
            kernel += kernel__of_one_batch(z_1[i], z_2[j], sigma=sigma)
    return kernel

def kernel_of_same_batch(z_1, z_2, sigma):
    N = z_1.size(0)
    kernel = 0
    for i in range(N):
        for j in range(N):
            if i != j:
                kernel += kernel__of_one_batch(z_1[i], z_2[j], sigma=sigma)
    return kernel

def calc_kernel(z_1, z_2, sigma):
    N = z_1.size(0)
    kernel = 0
    kernel += (1/(N*(N-1))) * (kernel_of_same_batch(z_1, z_1, sigma=sigma) + kernel_of_same_batch(z_2, z_2, sigma=sigma))
    kernel += (-2/(N*N)) * kernel_of_different_batch(z_1, z_2, sigma=sigma)
    return kernel

In [16]:
z_1 = torch.ones(100,50,1)
z_2 = torch.zeros(100,50,1)
sigma = 100
print(calc_kernel(z_1, z_2, sigma))

tensor(0.4424)


In [27]:
# Inverse Multiquadratics kernel
S = torch.tensor([0.1, 0.2, 0.5, 1.0, 2.0, 5.0, 10.0])
def IMK_of_one_batch(z_1, z_2, S):
    length, dim = z_1.size(0), z_1.size(1)
    z_1 = z_1.reshape(length*dim)
    z_2 = z_2.reshape(length*dim)#a
    l2_norm = torch.norm(z_1 - z_2)*torch.norm(z_1 - z_2)
    C = torch.rand(1)[0]*15.9 + 0.1
    kernel = 0
    for s in S:
        kernel += (s*C)/(s*C + l2_norm)
    return kernel

def IMK_of_different_batch(z_1, z_2, S):
    N = z_1.size(0)
    kernel = 0
    for i in range(N):
        for j in range(N):
            kernel += IMK_of_one_batch(z_1[i], z_2[j], S)
    return kernel

def IMK_of_same_batch(z_1, z_2, S):
    N = z_1.size(0)
    kernel = 0
    for i in range(N):
        for j in range(N):
            if i != j:
                kernel += IMK_of_one_batch(z_1[i], z_2[j], S)
    return kernel

def calc_IMK(z_1, z_2):
    S = torch.tensor([0.1, 0.2, 0.5, 1.0, 2.0, 5.0, 10.0])
    N = z_1.size(0)
    kernel = 0
    kernel += (1/(N*(N-1))) * (IMK_of_same_batch(z_1, z_1,S) + IMK_of_same_batch(z_2, z_2, S))
    kernel += (-2/(N*N)) * IMK_of_different_batch(z_1, z_2, S)
    return kernel
        
    

In [28]:
z_1 = torch.ones(5,50,1)
z_2 = torch.zeros(5,50,1)
one = torch.tensor(12.)
print(calc_IMK(z_1, z_2)+one)

tensor(23.0926)


In [24]:
torch.rand(1)[0]

tensor(0.3630)